## Charge parameters

In [1]:
import grid2op
import numpy as np
from lightsim2grid import LightSimBackend
from grid2op.Chronics import MultifolderWithCache
from l2rpn_baselines.utils import GymEnvWithRecoWithDN
from grid2op.Parameters import Parameters
from grid2op.utils import ScoreL2RPN2020
import torch
import datetime
import sys
import re
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from utils import *

from examples.ppo_stable_baselines.B_train_agent import CustomReward

2022-05-13 15:30:17.449701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-13 15:30:17.449724: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/boguslawskieva/miniconda3/envs/baselines_dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ENV_NAME = "l2rpn_wcci_2022_dev"

# Split sets and statistics parameters
is_windows = sys.platform.startswith("win32")
is_windows_or_darwin = sys.platform.startswith("win32") or sys.platform.startswith("darwin")
nb_process_stats = 4 if not is_windows_or_darwin else 1
deep_copy = is_windows  # force the deep copy on windows (due to permission issue in symlink in windows)
verbose = 1
SCOREUSED = ScoreL2RPN2020  # ScoreICAPS2021
name_stats = "_reco_powerline"

# Train parameters
env_name_train = '_'.join([ENV_NAME, "train"])
save_path = "./saved_model"
name = '_'.join(["GymEnvWithRecoWithDN", datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')])
gymenv_class = GymEnvWithRecoWithDN

In [2]:
def filter_chronics(x):
  list_chronics = ['2050-01-03_31', '2050-02-21_31', '2050-03-07_31', '2050-04-18_31'] # Names of chronics to keep
  p = re.compile(".*(" + '|'.join([c + '$' for c in list_chronics]) + ")")
  return re.match(p, x) is not None

In [3]:
with open("./preprocess_obs.json", "r", encoding="utf-8") as f:
    obs_space_kwargs = json.load(f)
with open("./preprocess_act.json", "r", encoding="utf-8") as f:
    act_space_kwargs = json.load(f)

In [6]:
p=Parameters()
p.LIMIT_INFEASIBLE_CURTAILMENT_STORAGE_ACTION = True

env_train = grid2op.make(ENV_NAME,
                   reward_class=CustomReward2,
                   backend=LightSimBackend(),
                   chronics_class=MultifolderWithCache,
                   param=p,
                   other_rewards={"CustomReward2": CustomReward2}
                   )
env_train.chronics_handler.real_data.set_filter(filter_chronics)
env_train.chronics_handler.real_data.reset()

nb_scenario = len(env_train.chronics_handler.real_data.available_chronics())

## Test Runner

In [ ]:
from grid2op.Runner import Runner
from examples.ppo_stable_baselines.C_evaluate_trained_model import load_agent

from grid2op.PlotGrid import PlotMatplot
plot_helper = PlotMatplot(observation_space=env_train.observation_space)

In [ ]:
agent_name = "GymEnvWithRecoWithDN_2022-05-12_21-37_learning_rate_3"
path_agent = os.path.join(save_path, agent_name)
path_results_agent = os.path.join("./model_results", agent_name)

In [ ]:
runner_params = env_train.get_params_for_runner()
runner_params["verbose"] = verbose
runner = Runner(**runner_params, agentClass=None, agentInstance=my_agent)

In [ ]:
os.makedirs(path_results_agent, exist_ok=True)

res = runner.run(path_save=path_results_agent, 
    nb_episode=nb_scenario, 
    nb_process=1, 
    max_iter=-1, 
    pbar=True,
    env_seeds=get_env_seed(ENV_NAME)[:nb_scenario], 
    # env_seeds=[42 for _ in range(nb_scenario)],
    agent_seeds=[0 for _ in range(nb_scenario)])